# Python for Data Science
## Three Ways to the Game of Life
### The Second Way


Here we are going to implement the Game of Life again, this time
using a slightly different model. The reason for doing this is twofold:

1. We are learning Python and it is great practice.
2. Stress the importance of step 1 in our *learning by doing*, meaning
   a review on the theory of our solution.
   
Here we will notice that using a slightly different model leads
to the choice of different primitives, and ends up having a dramatic
impact into the functionality of the program.



We are going to be referencing a notebook by Peter Norvig.

https://nbviewer.org/url/norvig.com/ipython/Life.ipynb

He has an interesting style, probably not the best for newer students,
so I re-implement some of that code here.